# Spring and Mass

We want to know the solution to the problem: $ F = m\cdot a$, making $a = -c/m\cdot v -k/m\cdot x + f(t)/m$, leading to:

$\ddot{x} = - \frac{c}{m}\cdot\dot{x} - \frac{k}{m} \cdot x + \frac{f(t)}{m}$

The state space representation of this problem is:
$$\underbrace{\begin{bmatrix}\ddot{x}\\
\dot{x}\end{bmatrix}}_{\dot{w}} = 
\underbrace{\begin{bmatrix} -c/m & -k/m \\
1 & 0\end{bmatrix}}_{A} \cdot \underbrace{\begin{bmatrix}\dot{x}\\
x\end{bmatrix}}_{w}+\underbrace{\begin{bmatrix}f(t)/m\\
0\end{bmatrix}}_{B}$$

We can define a function that outputs the change in $x$ and $\dot{x}$ as:

In [25]:
import numpy as np
import math
from systems.spring import Spring
from solvers.explicit import RK4, FE
from solvers.implicit import radauIA
#from utils.solve import solve, plot_sol, add_sol

In [71]:
# Solve and Plot Spring and Damper
# Initial Conditions
w_0 = np.array([0, 0.0])

# Parameters
t_span = [0,5]
steps = 1000
f_ext = lambda t,w: 0 if t<0.5 or t>2.5 else -200*9.81 #-100*math.cos(t)**2
solver2 = RK4()

k_vals = [1e5,1e5,1e5]
c_vals = [1e3,3e3,5e3]
# Solve and Plot
fig = None
for i in range(len(k_vals)):
    spring = Spring(k_vals[i],c_vals[i])
    legend=f'Sol {i+1} (k={spring.k:g}, c={spring.c:g})'
    t_sol, w_sol = spring.solve(solver2,w_0, t_span,251,f_ext=f_ext)
    if not fig: fig = spring.plot_sol(*spring.solve(solver2,w_0, t_span,251,f_ext=f_ext),["x[m]","v[m/s]"], legend=legend,
                title = "Simulation of Spring and Damper System with Weight added on t=0.5 and removed in t=2.5", plot=[1,0])
    else: spring.add_sol(fig, *spring.solve_fe(w_0,t_span,f_ext=f_ext), legend=legend, plot=[1,0])

import plotly.graph_objects as go
fig.add_trace(go.Scatter(x=t_sol,y=[f_ext(t,0)/1e5 for t in t_sol],legendgroup=1, name='External Force'))
fig.show()


# Box, Train and Rail System

In the box, train and Rail System we have two different elements (the falling box and the train) whose x and y position we can observe, and want to model the effect of actuators on the train that would make the difference between $x_{\text{box}}$ and $x_{\text{train}}$ 0 at the moment of landing.

For the box, $\dot{x} = 0$ and $\ddot{y} = -g$.

The train is located over an inclined rail which inflicts friction, such that there are three forces acting on the $x$-axis:

$F_{act} → \text{External Force from the Actuator}\\
F_{gx} → \text{Gravity Component on the x-axis}\\
F_{f} → \text{Friction from Contact with the Rail}$

Assuming that the actuator can produce diagonal forces, the function that governs movement on $x$ is then: $F_x=m\cdot a_x$, which leads to:

$a_x=-g\cdot \text{sin}(\theta) -\mu\cdot g\cdot \text{cos}(\theta)\cdot \text{sign}(\dot{x}) + \frac{\text{cos}(\theta)}{m}\cdot f(t)\\
a_y = -g\cdot \text{cos}(\theta) + \frac{\text{sin}(\theta)}{m}\cdot f(t)$

This is not a linear system, and we will not attempt to represent it as a state-space equation.

In [3]:
from systems.train_and_rail import Train_and_Rail
from controllers.pid import pid

In [4]:
controller = pid(30,10,1,error = lambda t, w: w[0]-w[3])
train = Train_and_Rail()
x_box_0 = np.random.random()*120
y_box_0 = 100
x_train_0 = 0
y_train_0 = 5+math.sin(train.theta)*x_train_0

w_0 = np.array([x_box_0,y_box_0,0,x_train_0,0,y_train_0])

t_sol, w_sol = train.solve_fe(w_0,[0,5], n_steps=250, controller=controller)

train.plot_animation(t_sol, w_sol, controller, SKIPFRAMES=5).show()